In [1]:
import os
import xarray
from datetime import datetime, timedelta
import numpy

In [2]:
WORKSPACE_FOLDER = os.path.join(os.path.abspath('..'), 'processing_record')
if not os.path.exists(WORKSPACE_FOLDER):
    os.makedirs(WORKSPACE_FOLDER)
ERA5_ALBEDO_FOLDER = '/data01/people/beichen/data_fd_et/ERA5_Albedo'

In [3]:
site_infos = [# site, lat, lon, UTC+
    ["CLC", 44.59666667, 123.4702778, 8],
    ["JRC", 31.80683056, 119.2172639, 8],
    ["PDC", 26.6, 106.3166667, 8],
    ["PJC", 40.94138889, 121.9602778, 8],
    ["HZF", 51.78111111, 121.0177778, 8],
    ["ZAG", 32.8, 102.55, 8],
    ["DBB", 41.64397222, 110.3314722, 8],
    ["HBG", 37.61666667, 101.3166667, 8],
    ["MPM", 22.4982, 114.0292, 8],
    ["BBY", 43.323, 141.8107, 9],
    ["TKY", 36.14616667, 137.4231111, 9],
    ["CRK", 38.20138889, 127.2505556, 9],
    ["GCK", 37.74833333, 127.1622222, 9],
    ["GDK", 37.74888889, 127.1491667, 9],
    ["SIP", 0.395204167, 102.7645506, 7],
    ["SDP", 0.699491389, 102.793305, 7],
    ["ASM", -22.2828, 133.2493, 9],
    ["CPW", -33.6152, 150.7236, 11],
    ["WAR", -43.095, 146.6545, 11]
]

In [4]:
def linear_interpolation_1h_10min(array):
#     array = array[::-1]
    h_10min = 6
    original_length = len(array)
    interpolated_length = 6 * original_length
    interpolated_array = numpy.zeros(interpolated_length)
    
    for i in range(original_length - 1):
        start_idx = i * h_10min
        end_idx = (i + 1) * h_10min
        interpolated_array[start_idx:end_idx] = numpy.linspace(array[i], array[i+1], h_10min, endpoint=False)
    
    # Repeat the last element to fill up the remaining slots
    interpolated_array[-h_10min:] = array[-1]
#     interpolated_array = interpolated_array[::-1]
    return interpolated_array

In [5]:
def read_site_era5_albedo_month(era5_albedo_grib, site_infos):
    albedo_ds = xarray.load_dataset(era5_albedo_grib, engine="cfgrib")
    
    site_albedo_list = []
    for site_info in site_infos:
        lat_site = site_info[1]
        lon_site = site_info[2]
        albedo_site_ds = albedo_ds.interp(longitude=[lon_site], latitude=[lat_site], method="nearest", kwargs={"fill_value": "extrapolate"}) # linear

        albedo_v = numpy.array(albedo_site_ds.to_array()[0,1:,:,0,0])
        albedo_v_1h = albedo_v.flatten()
        albedo_v_1h = numpy.insert(albedo_v_1h, 0, albedo_site_ds.to_array()[0,0,23,0,0])
        albedo_v_1h = numpy.delete(albedo_v_1h, -1)
        
        site_albedo_list.append(albedo_v_1h)
    
    return site_albedo_list

In [6]:
# 1. record full day from 20171201 to 20191231 in hourly
# 2. extend hourly into every 10-min 
# 3. save site data according to localtime with UTC offset

time_internal = 10 # min
START_TIME = '2018-01-01T00:00:00Z' # local time
END_TIME = '2019-12-31T23:59:59Z'

data_record = [] # hourly
for site_idx in range(len(site_infos)):
    data_record.append([])
    
years = ['2017', '2018', '2019']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

for year_str in years:
    for month_str in months:
        year_month = year_str + '_' + month_str
        grib_filename = os.path.join(ERA5_ALBEDO_FOLDER, year_month + '.grib')
        if os.path.exists(grib_filename):
            file_site_v_list = read_site_era5_albedo_month(grib_filename, site_infos)
            for current_site_idx in range(len(site_infos)):
                site_data_record = data_record[current_site_idx]
                for site_v in file_site_v_list[current_site_idx]:
                    site_data_record.append(site_v)
                data_record[current_site_idx] = site_data_record
            print(year_month)

/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to

/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to

/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to

2017_12


/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to

...

/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to

/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to

/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to

2019_12


/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/data01/people/beichen/anaconda3/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to

In [7]:
local_start_date = datetime.strptime(START_TIME, "%Y-%m-%dT%H:%M:%SZ")
local_end_date = datetime.strptime(END_TIME, "%Y-%m-%dT%H:%M:%SZ")

data_start_date = datetime.strptime('2017_12-01T00:00:00Z', "%Y_%m-%dT%H:%M:%SZ")
for current_site_idx in range(len(site_infos)):
    site_name = site_infos[current_site_idx][0]
    site_utcOffset = site_infos[current_site_idx][3]
    site_data_record = data_record[current_site_idx]
    site_data_record_10min = linear_interpolation_1h_10min(site_data_record)
    
    local_current_date = data_start_date + timedelta(hours=site_utcOffset)    
    albedo_site_data = []
    for site_v_10min in site_data_record_10min:
        if local_current_date >= local_start_date and local_current_date < local_end_date:
            local_current_time = local_current_date.strftime("%Y-%m-%dT%H:%M:%SZ")
            albedo_site_data.append([local_current_time, str(site_v_10min)])
        local_current_date = local_current_date + timedelta(minutes=time_internal)
        
    albedo_site_data = numpy.array(albedo_site_data)
    save_filename = os.path.join(WORKSPACE_FOLDER, site_name+'_ERA5_Albedo'+'.csv')
    print(save_filename)
    numpy.savetxt(save_filename, albedo_site_data, delimiter=",", fmt='%s')

/data01/people/beichen/workspace/20240320/processing_record/CLC_ERA5_Albedo.csv
/data01/people/beichen/workspace/20240320/processing_record/JRC_ERA5_Albedo.csv
/data01/people/beichen/workspace/20240320/processing_record/PDC_ERA5_Albedo.csv
/data01/people/beichen/workspace/20240320/processing_record/PJC_ERA5_Albedo.csv
/data01/people/beichen/workspace/20240320/processing_record/HZF_ERA5_Albedo.csv
/data01/people/beichen/workspace/20240320/processing_record/ZAG_ERA5_Albedo.csv
/data01/people/beichen/workspace/20240320/processing_record/DBB_ERA5_Albedo.csv
/data01/people/beichen/workspace/20240320/processing_record/HBG_ERA5_Albedo.csv
/data01/people/beichen/workspace/20240320/processing_record/MPM_ERA5_Albedo.csv
/data01/people/beichen/workspace/20240320/processing_record/BBY_ERA5_Albedo.csv
/data01/people/beichen/workspace/20240320/processing_record/TKY_ERA5_Albedo.csv
/data01/people/beichen/workspace/20240320/processing_record/CRK_ERA5_Albedo.csv
/data01/people/beichen/workspace/2024032